<a href="https://colab.research.google.com/github/zZHugoZz/Machine-learning-Pytorch/blob/main/Male_female_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import random
from typing import Callable
from pathlib import Path
from PIL import Image
import torch
from torch import nn
from torch.optim import Optimizer
from torch.nn.modules.loss import _Loss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms as T
from torchinfo import summary, ModelStatistics
import matplotlib.pyplot as plt
import numpy as np

In [62]:
SEED = 5432
BATCH_SIZE = 32

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [63]:
root_path = Path("drive/MyDrive/Machine learning")
datasets_path = root_path.joinpath("Datasets/Male and Female face dataset")

In [64]:
transform = T.Compose([
    T.Resize((224, 224)),
    T.RandomCrop((224, 224)),
    T.ColorJitter(0.5, 0.2, 0.5, 0.3),
    T.RandomRotation(25),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.5108, 0.4549, 0.4235], [0.2577, 0.2461, 0.2408])
])

In [65]:
dataset = ImageFolder(str(datasets_path), transform)

In [66]:
class FacesDataset(Dataset):
    def __init__(
        self,
        dataset: ImageFolder,
        train_size: float = 0.8
    ) -> None:
        self.dataset = dataset
        self.train_size = train_size

    def __getitem__(self) -> tuple[Dataset, Dataset]:
        train_split = int(len(self.dataset) * self.train_size)
        test_split = len(self.dataset) - train_split
        train_data, test_data = random_split(self.dataset, [train_split, test_split])
        return train_data, test_data

    def __len__(self) -> int:
        return len(self.dataset)

    def get_classes(self) -> list[str]:
        return self.dataset.classes


faces_dataset = FacesDataset(dataset)
train_data, test_data = faces_dataset.__getitem__()

In [67]:
train_dataloader = DataLoader(train_data, BATCH_SIZE, True, pin_memory=True)
test_dataloader = DataLoader(test_data, BATCH_SIZE, True, pin_memory=True)

In [68]:
def compute_mean_std(
    dataloader: DataLoader,
) -> tuple[torch.Tensor | float, torch.Tensor | float]:
    global_mean = 0.0
    global_std = 0.0

    for images, _ in dataloader:
        mean = 0.0
        std = 0.0

        for image in images:
            mean += image.mean((1, 2))
            std += image.std((1, 2))

        mean /= len(images)
        std /= len(images)

        global_mean += mean
        global_std += std

    global_mean /= len(dataloader)
    global_std /= len(dataloader)

    return global_mean, global_std

In [69]:
class ModelPerformanceStats:
    def __init__(self, epoch_values: list) -> None:
        self.epoch_values = epoch_values
        self.fig, (self.loss_ax, self.acc_ax) = plt.subplots(1, 2, figsize=(10, 3))

    def plot_loss(self, train_loss_values: list, test_loss_values: list) -> None:
        self.loss_ax.plot(self.epoch_values, train_loss_values, label="train loss")
        self.loss_ax.plot(self.epoch_values, test_loss_values, label="test loss")
        self.loss_ax.set_title("Loss")
        self.loss_ax.grid()
        self.loss_ax.legend(loc="upper left")

    def plot_accuracy(self, train_acc_values: list, test_acc_values: list) -> None:
        self.acc_ax.plot(self.epoch_values, train_acc_values, label="train accuracy")
        self.acc_ax.plot(self.epoch_values, test_acc_values, label="test accuracy")
        self.acc_ax.set_title("Accuracy")
        self.acc_ax.grid()
        self.acc_ax.legend(loc="upper left")

    def show(self) -> None:
        self.fig.tight_layout()
        self.fig.show()

In [70]:
def plot_images(n_rows: int, n_cols: int, datasets_path: Path) -> None:
    unormalized_transform = T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
    ])
    data = ImageFolder(str(datasets_path), transform=unormalized_transform)
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 10))

    for i, ax in enumerate(axs.flat):
        image, label = data[random.randint(0, len(data))]
        ax.imshow(image.permute(1, 2, 0))
        ax.axis("off")
        ax.set_title(data.classes[label])

    fig.tight_layout()
    fig.show()

In [71]:
def plot_predictions(
    n_rows: int, n_cols: int, preds: torch.Tensor, images: torch.Tensor, dataset: ImageFolder
) -> None:
    fig, axs = plt.subplots(n_rows, n_cols, figsize=(10, 10))

    for i, ax in enumerate(axs.flat):
        ax.imshow(images[i].permute(1, 2, 0))
        ax.axis("off")
        ax.set_title(f"predicted: {dataset.classes[preds[i]]}")

    fig.tight_layout()
    fig.show()

In [72]:
def get_summary(model: nn.Module, input_size: tuple[int, ...]) -> ModelStatistics:
    return summary(model, input_size)


def save_model(model_state_dict: object, path: str, model_file_name: str) -> None:
    file = f"{path}/{model_file_name}"
    torch.save(model_state_dict, file)


def validate_model(path: str, model: nn.Module, validation_data: torch.Tensor) -> None:
    model.load_state_dict(torch.load(path))
    model.eval()
    logits = model(validation_data.to(device))
    preds = torch.softmax(logits, dim=1).argmax(dim=1)
    preds.cpu()
    plot_predictions(4, 4, preds, validation_data, dataset)

In [ ]:
class FaceClassifier(nn.Module):
    def __init__(self, in_features: int, out_features: int, hidden_units: int = 10) -> None:
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.hidden_units = hidden_units

        self.block_1 = nn.Sequential(
            nn.Conv2d(in_features, hidden_units, 1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units * 55 * 55, out_features)
        )
        self.apply(self.weight_init)

    def weight_init(self, m: nn.Module) -> None:
        if isinstance(m, nn.Conv2d):
            nn.init.xavier_normal_(m.weight, nn.init.calculate_gain("relu"))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.fc(x)
        return x


face_classifier = FaceClassifier(3, 2)
get_summary(face_classifier, (32, 3, 224, 224))

In [74]:
class TrainAndTest:
    def __init__(
        self,
        model: nn.Module,
        train_data: DataLoader,
        test_data: DataLoader,
        criterion: _Loss,
        optimizer: Optimizer,
        device: str,
        scheduler: ReduceLROnPlateau | None = None
    ) -> None:
        self.model = model
        self.train_data = train_data
        self.test_data = test_data
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.scheduler = scheduler

    def train_step(self) -> tuple[float, float]:
        train_loss = torch.tensor(0, dtype=torch.float32).to(device)
        train_acc = torch.tensor(0, dtype=torch.float32).to(device)

        self.model.train()
        for X, y in self.train_data:
            X, y = X.to(self.device), y.to(self.device)
            preds = self.model(X)

            loss = self.criterion(preds, y)
            acc = self.get_accuracy(torch.softmax(preds, dim=1).argmax(dim=1), y)
            train_loss += loss
            train_acc += acc

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        if self.scheduler is not None:
            self.scheduler.step(train_loss)

        train_loss /= len(self.train_data)
        train_acc /= len(self.train_data)
        print(f"train loss: {train_loss:.5f} | train accuracy: {train_acc:.2f}%")
        return train_loss.item(), train_acc.item()

    def test_step(self) -> tuple[float, float]:
        test_loss = torch.tensor(0, dtype=torch.float32).to(device)
        test_acc = torch.tensor(0, dtype=torch.float32).to(device)

        self.model.eval()
        with torch.inference_mode():
            for X, y in self.test_data:
                X, y = X.to(self.device), y.to(self.device)
                preds = self.model(X)

                loss = self.criterion(preds, y)
                acc = self.get_accuracy(torch.softmax(preds, dim=1).argmax(dim=1), y)
                test_loss += loss
                test_acc += acc

            test_loss /= len(self.test_data)
            test_acc /= len(self.test_data)
            print(f"test loss: {test_loss:.5f} | test accuracy: {test_acc:.2f}%\n")
            return test_loss.item(), test_acc.item()

    def get_accuracy(self, preds: torch.Tensor, labels: torch.Tensor) -> float:
        equals = torch.eq(preds, labels).sum().item()
        acc = (equals / len(labels)) * 100
        return acc

In [ ]:
EPOCHS = 50
LEARNING_RATE = 0.001

total_steps = len(train_dataloader) * EPOCHS

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(face_classifier.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5)

epoch_values = []
train_loss_values = []
test_loss_values = []
train_acc_values = []
test_acc_values = []

train_test = TrainAndTest(
    face_classifier,
    train_dataloader,
    test_dataloader,
    criterion,
    optimizer,
    device,
    scheduler,
)

for epoch in range(EPOCHS):
    print(f"EPOCH {epoch}\n----------")
    train_loss, train_acc = train_test.train_step()
    test_loss, test_acc = train_test.test_step()

    epoch_values.append(epoch)
    train_loss_values.append(train_loss)
    test_loss_values.append(test_loss)
    train_acc_values.append(train_acc)
    test_acc_values.append(test_acc)

save_model(
    face_classifier.state_dict(),
    "drive/MyDrive/Machine learning/Models",
    "face_classifier"
)

plot_performances = ModelPerformanceStats(epoch_values)
plot_performances.plot_loss(train_loss_values, test_loss_values)
plot_performances.plot_accuracy(train_acc_values, test_acc_values)
plot_performances.show()

In [76]:
from torchvision.io import read_image


test_path = Path("/content/drive/MyDrive/Machine learning/Datasets/Male and Female face dataset/Test images")
test_transforms = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
])

images = [
    test_transforms(read_image(str(image_path)))
    for image_path in test_path.glob("*")
    if image_path.suffix.lower() in [".jpeg", ".jpg", ".png"]
]
images = [image for image in images if image.shape != torch.Size((4, 224, 224))]
images = torch.stack(images)

In [ ]:
model_path = "/content/drive/MyDrive/Machine learning/Models/face_classifier"

loaded_model = face_classifier
loaded_model.load_state_dict(torch.load(model_path, torch.device(device)))
logits = loaded_model(images)
preds = torch.softmax(logits, dim=1).argmax(dim=1)

plot_predictions(2, 4, preds, images, dataset)